## SSH key pair generation

we create a SSH key pair that will be used to connect to the deployed node.

In [ ]:
%x(ssh-keygen -b 2048 -t rsa -f /tmp/sshkey -q -N "")

## Job and deployment definition

In [ ]:
require 'xp5k'
myxp = XP5K::XP.new
myxp.define_job({
    :resources => ["nodes=1, walltime=1:00:00"],
    :site      => 'lyon',
    :types     => ["deploy"],
    :name      => "iruby notebook" ,
    :roles     => [XP5K::Role.new({:name => 'mynode', :size => 1})],
    :command   => "sleep 86400"
    })
myxp.define_deployment({
    :site           => 'lyon',
    :environment    => "jessie-x64-nfs",
    :roles          => %w{ mynode },
    :key            => File.read("/tmp/sshkey.pub")
})

## Reservation and deployment of the environment

In [ ]:
myxp.submit
myxp.wait_for_jobs
myxp.deploy

# Launch a command

In [ ]:
require 'xp5k/rake'
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
    cmd = ['uptime -s']
end

## Clean the experiment

In [ ]:
myxp.clean